In [79]:
import re
import json
import stanza
import argparse

import numpy as np
import pandas as pd

from itertools import chain
from ast import literal_eval
from collections import defaultdict

from functions_score_section import read_alignments, score_vec_rslts_chapter_level, \
build__src_2_tgt_dict, build_tgt_2_src_dict, score_fr_sents

In [8]:
# get vecalign results
vec_rslts_path = "/home/craig.car/repos/chiron/align_texts_project/data/lucretius/lucretius_en1893_vecrslts"
vec_rslts = read_alignments(vec_rslts_path)

In [9]:
lat_dict_path = "/home/craig.car/repos/chiron/align_texts_project/data/lucretius/lat_sent2book_dict.json"

with open(lat_dict_path) as f:
    lat_sent2book_name = json.load(f)

en1893_dict_path = "/home/craig.car/repos/chiron/align_texts_project/data/lucretius/en1893_sent2section_dict.json"

with open(en1893_dict_path) as f:
    en1893_sent2section_name = json.load(f)
    

In [57]:
# get peritext sections in en1893
extraneous_sections = []
for item in list(en1893_sent2section_name.values()):
    if isinstance(item, str):
        extraneous_sections.append(item)
    else:
        extraneous_sections.extend(item)

In [58]:
extraneous_sections = np.unique(np.array(extraneous_sections))

# Score vecalign results: by prediction

In [59]:
tp_strict_, tp_lax_, overlaps_, errors_, correct_nulls_ = score_vec_rslts_chapter_level(
    vec_rslts, lat_sent2book_name, en1893_sent2section_name, extraneous_sections)

In [63]:
print(tp_strict_)
print(tp_lax_)
print(overlaps_)
print(correct_nulls_)

8146
0
[]
8146


In [66]:
len(vec_rslts)

10534

# Analyze vecalign results: by French sentences

In [65]:
# get dict of tgt sentences' alignments to src sents
tgt_sent_2_src_sent_aligns = build_tgt_2_src_dict(vec_rslts)

In [69]:
# test
keys = list(tgt_sent_2_src_sent_aligns.keys())
keys = sorted(keys)
keys == [x for x in range(0, 13966)]

True

In [80]:
rslts_en1893_sents = score_fr_sents(tgt_sent_2_src_sent_aligns, en1893_sent2section_name,
                   lat_sent2book_name, extraneous_sections)

el aligned sents is {'null'}
fr chapter is forewordbook0title
el aligned sents is {'null'}
fr chapter is forewordbook0title
el aligned sents is {'null'}
fr chapter is forewordbook0title
el aligned sents is {'null'}
fr chapter is forewordbook0title
el aligned sents is {'null'}
fr chapter is forewordbook0title
el aligned sents is {'null'}
fr chapter is forewordbook0title
el aligned sents is {'null'}
fr chapter is forewordbook0title
el aligned sents is {'null'}
fr chapter is forewordbook0title
el aligned sents is {'null'}
fr chapter is forewordbook0title
el aligned sents is {'null'}
fr chapter is forewordbook0title
el aligned sents is {'null'}
fr chapter is forewordbook0title
el aligned sents is {'null'}
fr chapter is forewordbook0title
el aligned sents is {'null'}
fr chapter is forewordbook0title
el aligned sents is {'null'}
fr chapter is forewordbook0title
el aligned sents is {'null'}
fr chapter is forewordbook0title
el aligned sents is {'null'}
fr chapter is forewordbook0title
el align

ValueError: shape mismatch: objects cannot be broadcast to a single shape.  Mismatch is between arg 0 with shape (48,) and arg 1 with shape (2,).

In [71]:
rslts_en1893_sents

NameError: name 'rslts_en1893_sents' is not defined